# GEEMAP ONLINE AND OFFLINE DATA ACCESS 
## (PART 1 - SINGLE BAND SPATIAL DATA)

Course: CEE5003 Remote Sensing of Land Surfaces  

Created by: Alfonso Torres  

Date: May 2024

***

### Goal 

Demonstrate data access and retrieval when using Google Earth Engine 
***

### Data Sources

* Cloud:  
 a. GEE [NLDAS-2: North American Land Data Assimilation System Forcing Fields](https://developers.google.com/earth-engine/datasets/catalog/NASA_NLDAS_FORA0125_H002#description)    

   Products of interest

   precipitation "total_precipitation"	  
   air temperature at 2 meters "temperature"  
   evapotranspiration "potential_evaporation"

 b. GEE [TIGER: US Census States 2018](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States)  




* Local:
    None  

***

### Implementation

1. Importing modules

In [113]:
%matplotlib inline

import ee
import geemap
import eemont
import pprint

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [ ]:
##### uncomment this cell and run it if the cell below fails
# import geemap
# m = geemap.Map()
# m

2. Initialize Earth Engine

In [ ]:
ee.Initialize()

### Accessing Different Raster Products

#### Climate and Weather

3. Load Utah Boundary

In [ ]:
boundary_ut =ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Utah')).first()

In [ ]:
#boundary_ut

4. Import NLDAS Data 

In [ ]:
NLDAS_daily = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")

5. Clip ERAS5 products of interest for the most recent April 1st

In [ ]:
#filtering by date: April 1st.
date = '2024-05-15'

In [ ]:
pp_ut = NLDAS_daily.closest(date).select('total_precipitation') .first()

pp_ut = pp_ut.clip(boundary_ut)

In [ ]:
temp_ut = NLDAS_daily.closest(date).select('temperature') .first()

temp_ut = temp_ut.clip(boundary_ut)

In [ ]:
et0_ut = NLDAS_daily.closest(date).select('potential_evaporation') .first()

et0_ut = et0_ut.clip(boundary_ut)

6. To plot single rasters, we need their min and max values. Creating a function to do that.

In [ ]:
# creating a function to estimate mins and maxs of an image

def min_max_values(single_band_image):
# getting the min and max value of a single band image
    meanDictionary = single_band_image.reduceRegion(**{
    'reducer': ee.Reducer.minMax(), #there are multiple reducers, this extract the min and max of an image
    'geometry': single_band_image.geometry(),
    'bestEffort':True
    })
# reduced values from an image area are reported in a python dictionary
    max_min_values = meanDictionary.getInfo()
    keys_list = list(max_min_values.keys())
#extracting the keys names by index order from dictionary
    max_key = keys_list[0]
    min_key = keys_list[1]
# extracting values by key names    
    maxValue = meanDictionary.get(max_key).getInfo()
    minValue = meanDictionary.get(min_key).getInfo()

    #rounding the numbers
    maxValue = round(maxValue)
    minValue = round(minValue)

    return (maxValue, minValue)

7. Creating a Map

In [ ]:
Map1 = geemap.Map()

Map1.centerObject(boundary_ut, 6)	


8. Preparing and displaying ERAS5 precipitation

In [ ]:
# getting the min and max value
max_min_pp=min_max_values(pp_ut)

# setting the colorbar from Colorbrewer
color_range = ['#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6','#2171b5','#08519c','#08306b']

# setting up the value and color range 
singleBandVis = {
    "min": max_min_swe[1],
    "max": max_min_swe[0],
    "palette": color_range
}

# adding a colorbar in the map
palette = singleBandVis["palette"]
vmin = singleBandVis["min"]
vmax = singleBandVis["max"]

Map1.addLayer(pp_ut, singleBandVis, 'UT precipitation (kg/m^2)')
Map1.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax, layer_name="UT precipitation (kg/m^2)")

In [ ]:

Map1

9. Prepapring and displaying NLDAS air temperature

In [ ]:
# getting the min and max value
max_min_temp=min_max_values(temp_ut)

# setting the colorbar from Colorbrewer
color_range = ['#ffffcc','#ffeda0','#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#bd0026','#800026']

# setting up the value and color range 
singleBandVis = {
    "min": max_min_temp[1],
    "max": max_min_temp[0],
    "palette": color_range
}

# adding a colorbar in the map
palette = singleBandVis["palette"]
vmin = singleBandVis["min"]
vmax = singleBandVis["max"]

Map1.addLayer(temp_ut, singleBandVis, 'UT air temp at 2m (°C)')
Map1.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax, layer_name="UT air temp at 2m (°C)")

10. Preparing and dispaying NLDAS potential evapotranspiration

In [ ]:
# getting the min and max value
max_min_et=min_max_values(et0_ut)

# setting the colorbar from Colorbrewer
color_range = ['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494','#081d58']

# setting up the value and color range 
singleBandVis = {
    "min": max_min_et[1],
    "max": max_min_et[0],
    "palette": color_range
}

# adding a colorbar in the map
palette = singleBandVis["palette"]
vmin = singleBandVis["min"]
vmax = singleBandVis["max"]

Map1.addLayer(et0_ut, singleBandVis, 'UT evapotranspiration (kg/m^2)')
Map1.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax, layer_name="UT evapotranspiration (kg/m^2)")